In [1]:
import warnings

import numpy as np
import pandas as pd
import useful_rid_code as rid
from openpyxl import load_workbook

warnings.simplefilter("ignore")

# RID 

## Fill-in OSPAR reporting template (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 1991


## 1. Get summary data

The notebooks `summary_table_{year}.ipynb` calculate summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [4]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)
del mon_df["new_rid_group"]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "old_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("old_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "old_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"], mon_df["mean_q_1000m3/day"]

# Convert units
mon_df["Hg_kg"] = mon_df["Hg_kg"] / 1000.0  # kg to tonnes
mon_df["NH4-N_tonnes"] = mon_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["NO3-N_tonnes"] = mon_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTN_tonnes"] = mon_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTP_tonnes"] = mon_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["PO4-P_tonnes"] = mon_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["SPM_tonnes"] = mon_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

As   Cd    Cr    Cu   Hg  NO3-N  PO4-P  \
ospar_region        old_rid_group                                             
LOFOTEN-BARENTS SEA rid_108         1.0  0.0   5.0  11.0  0.0    0.0    0.0   
                    rid_11          1.0  0.0   1.0   4.0  0.0    0.0    0.0   
                    rid_36          1.0  0.0   5.0  26.0  0.0    0.0    0.0   
NORTH SEA           rid_108         1.0  1.0   7.0  17.0  0.0    2.0    0.0   
                    rid_11          0.0  0.0   1.0   2.0  0.0    1.0    0.0   
                    rid_36          2.0  0.0   3.0  20.0  0.0    4.0    0.0   
NORWEGIAN SEA2      rid_108         1.0  0.0   9.0  32.0  0.0    1.0    0.0   
                    rid_11          1.0  0.0   3.0  29.0  0.0    1.0    0.0   
                    rid_36          3.0  2.0  11.0  33.0  0.0    2.0    0.0   
SKAGERAK            rid_108         1.0  0.0   2.0   7.0  0.0    1.0    0.0   
                    rid_11          9.0  1.0   3.0  63.0  0.0   11.0    0.0   
                    rid_36          1.0  1.0   2.0  11.0  0.0    2.0    0.0   
NORWAY              rid_108         4.0  2.0  23.0  67.0  0.0    5.0    0.0   
                    rid_11         11.0  1.0   7.0  97.0  0.0   13.0    0.0   
                    rid_36          8.0  3.0  21.0  90.0  0.0    7.0    0.0   

                                     Pb    SPM       TOC  TOTN  TOTP     Zn  \
ospar_region        old_rid_group                                             
LOFOTEN-BARENTS SEA rid_108         4.0   17.0   17960.0   1.0   0.0   49.0   
                    rid_11          0.0    5.0   11028.0   1.0   0.0    6.0   
                    rid_36          8.0   34.0   35690.0   2.0   0.0   43.0   
NORTH SEA           rid_108         5.0   13.0   16653.0   4.0   0.0   67.0   
                    rid_11          1.0    4.0    3059.0   1.0   0.0   11.0   
                    rid_36          8.0   14.0   29263.0   5.0   0.0   81.0   
NORWEGIAN SEA2      rid_108         7.0   18.0   16554.0   2.0   0.0   47.0   
                    rid_11          2.0   16.0   16763.0   1.0   0.0   72.0   
                    rid_36         13.0   69.0   58658.0   4.0   0.0   76.0   
SKAGERAK            rid_108         2.0    8.0   11535.0   2.0   0.0   37.0   
                    rid_11         18.0  165.0  123170.0  18.0   1.0  259.0   
                    rid_36          5.0    8.0   25380.0   3.0   0.0   82.0   
NORWAY              rid_108        18.0   56.0   62701.0   9.0   0.0  199.0   
                    rid_11         21.0  189.0  154021.0  21.0   1.0  348.0   
                    rid_36         35.0  125.0  148991.0  14.0   0.0  282.0   

                                    Ag  NH4-N    Ni      SiO2  
ospar_region        old_rid_group                              
LOFOTEN-BARENTS SEA rid_108        0.0    0.0   4.0   20810.0  
                    rid_11         0.0    0.0   1.0   12801.0  
                    rid_36         0.0    0.0  29.0   54826.0  
NORTH SEA           rid_108        0.0    0.0   4.0   18169.0  
                    rid_11         0.0    0.0   1.0    2522.0  
                    rid_36         0.0    0.0   4.0   24185.0  
NORWEGIAN SEA2      rid_108        0.0    0.0   4.0   13467.0  
                    rid_11         0.0    0.0   4.0   12077.0  
                    rid_36         0.0    0.0  16.0   35175.0  
SKAGERAK            rid_108        0.0    0.0   2.0    9164.0  
                    rid_11         0.0    1.0  30.0  114668.0  
                    rid_36         0.0    0.0   2.0   14772.0  
NORWAY              rid_108        0.0    0.0  15.0   61610.0  
                    rid_11         0.0    1.0  37.0  142068.0  
                    rid_36         0.0    1.0  51.0  128958.0

### 1.2. Unmonitored areas

In [5]:
# Read data
in_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Add missing columns if necessary
cols = ["sew_Hg", "sew_S.P.M.", "ind_Hg"]
for col in cols:
    if col not in umon_df.columns:
        umon_df[col] = 0

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,359069.0,0.0,0.0,2962.0,156.0,0.0,0.0,28670.0,610.0,0.0,...,0.0,4.0,2.0,22.0,95.0,5.0,0.0,46.0,0,0
LOFOTEN-BARENTS SEA,112485.0,0.0,0.0,0.0,0.0,0.0,0.0,4975.0,92.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
NORTH SEA,112678.0,0.0,0.0,132.0,0.0,0.0,0.0,11051.0,183.0,0.0,...,0.0,3.0,2.0,4.0,92.0,1.0,0.0,38.0,0,0
NORWEGIAN SEA2,125831.0,0.0,0.0,909.0,65.0,0.0,0.0,9439.0,225.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
SKAGERAK,8076.0,0.0,0.0,1921.0,91.0,0.0,0.0,3205.0,109.0,0.0,...,0.0,1.0,0.0,18.0,3.0,4.0,0.0,8.0,0,0


### 1.3. Loads for 11 main rivers

In [6]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df["new_rid_group"]

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df["station_code"], rid11_df["station_name"]
del rid11_df["old_rid_group"], rid11_df["ospar_region"]
del rid11_df["mean_q_1000m3/day"]

cols = [i for i in rid11_df.columns if i.split("_")[1] != "Est"]
rid11_df = rid11_df[cols]

# Convert units
rid11_df["Hg_kg"] = rid11_df["Hg_kg"] / 1000.0  # kg to tonnes
rid11_df["NH4-N_tonnes"] = rid11_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["NO3-N_tonnes"] = rid11_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTN_tonnes"] = rid11_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTP_tonnes"] = rid11_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["PO4-P_tonnes"] = rid11_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["SPM_tonnes"] = rid11_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split("_")[0] for i in rid11_df.columns]

rid11_df.head()

,As,Cd,Cr,Cu,Hg,NO3-N,PO4-P,Pb,SPM,TOC,TOTN,TOTP,Zn,Ag,NH4-N,Ni,SiO2
station_id,,,,,,,,,,,,,,,,,
29612,1.715655,0.144235,0.000000,8.286101,0.008091,1.727816,0.012374,2.084903,11.583425,16110.439028,2.672233,0.038000,34.784293,0.007506,0.134868,4.254869,17802.615702
29779,0.760874,0.076306,1.295309,3.825684,0.006031,0.146483,0.005991,0.000000,4.541410,11028.283598,0.629038,0.024852,5.913326,0.004196,0.015043,1.217377,12800.686320
29821,0.186476,0.074613,0.478470,1.989670,0.000000,0.355654,0.001244,0.746126,1.939929,2120.933419,0.616798,0.012435,9.948352,0.006883,0.029654,0.562358,2152.014169
29782,0.639115,0.233407,2.055291,6.932919,0.003738,0.338273,0.009642,1.719623,12.365822,10347.221014,0.748482,0.021287,27.249138,0.006967,0.056595,2.613047,7523.571247
36225,0.069381,0.051713,0.131006,3.447520,0.000000,0.118939,0.008964,3.102768,8.101672,962.337344,0.386467,0.031890,5.171280,0.001737,0.026552,0.178409,1185.238967


## 2. Fill-in template

The template is usually sent each year by Csilla at NIBIO. However, it doesn't seem to change, so old versions can also be used e.g. here:

    ../../../Results/OSPAR/Blank_Template

In [7]:
# Copy of template to update
temp_path = rid.copy_ospar_template(year)

In [8]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """Update the OSPAR template for point source data.

    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "S.P.M.",
        "TOC": "TOC",
        "PO4-P": "po4",
        "P-Total": "p",
        "NO3-N": "no3",
        "NH4-N": "nh4",
        "N-Total": "n",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names
    names_dict = {
        "Norwegian Sea (NO)": "NORWEGIAN SEA2",
        "Barents Sea (NO)": "LOFOTEN-BARENTS SEA",
        "Skagerrak (NO)": "SKAGERAK",
        "North Sea (NO)": "NORTH SEA",
        "Norway Total": "NORWAY",
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            try:
                val = df.loc[names_dict[reg], "%s_%s" % (src, par_dict[par])]
            except KeyError:
                val = 0

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [9]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split("_")[0] == "sew"]
sew_df = umon_df[cols].copy()

# Convert units
sew_df["sew_Hg"] = sew_df["sew_Hg"] / 1000.0  # kg to tonnes
sew_df["sew_nh4"] = sew_df["sew_nh4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_no3"] = sew_df["sew_no3"] / 1000.0  # tonnes to ktonnes
sew_df["sew_n"] = sew_df["sew_n"] / 1000.0  # tonnes to ktonnes
sew_df["sew_po4"] = sew_df["sew_po4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_p"] = sew_df["sew_p"] / 1000.0  # tonnes to ktonnes
sew_df["sew_S.P.M."] = sew_df["sew_S.P.M."] / 1000.0  # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,
NORWAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOFOTEN-BARENTS SEA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NORTH SEA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NORWEGIAN SEA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SKAGERAK,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Update sheet 5a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
]

update_spreadsheet_point_sources(temp_path, "5a", pars, "sew", sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [11]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split("_")[0] == "ind"]
ind_df = umon_df[cols].copy()

# Convert units
ind_df["ind_Hg"] = ind_df["ind_Hg"] / 1000.0  # kg to tonnes
ind_df["ind_nh4"] = ind_df["ind_nh4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_no3"] = ind_df["ind_no3"] / 1000.0  # tonnes to ktonnes
ind_df["ind_n"] = ind_df["ind_n"] / 1000.0  # tonnes to ktonnes
ind_df["ind_po4"] = ind_df["ind_po4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_p"] = ind_df["ind_p"] / 1000.0  # tonnes to ktonnes
ind_df["ind_S.P.M."] = ind_df["ind_S.P.M."] / 1000.0  # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.962,0.155525,0.093315,0.14810,2.22150,1622.1882,395.7,0.485,3.9523,1.925,21.9958,95.169,5.456,0.4,0.04648
LOFOTEN-BARENTS SEA,0.000,0.000000,0.000000,0.00000,0.00000,1600.0000,0.0,0.000,0.0000,0.000,0.0000,0.000,0.000,0.0,0.00000
NORTH SEA,0.132,0.000283,0.000170,0.00660,0.09900,10.1290,178.6,0.000,3.1630,1.900,3.9980,92.244,1.300,0.4,0.03810
NORWEGIAN SEA2,0.909,0.064620,0.038772,0.04545,0.68175,5.1020,140.0,0.000,0.0000,0.000,0.0000,0.000,0.000,0.0,0.00000
SKAGERAK,1.921,0.090622,0.054373,0.09605,1.44075,6.9572,77.1,0.485,0.7893,0.025,17.9978,2.925,4.156,0.0,0.00838


In [12]:
# Update sheet 5b
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5b", pars, "ind", ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [13]:
# Get fish data
cols = [i for i in umon_df.columns if i.split("_")[0] == "fish"]
fish_df = umon_df[cols].copy()

# Convert units
fish_df["fish_nh4"] = fish_df["fish_nh4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_no3"] = fish_df["fish_no3"] / 1000.0  # tonnes to ktonnes
fish_df["fish_n"] = fish_df["fish_n"] / 1000.0  # tonnes to ktonnes
fish_df["fish_po4"] = fish_df["fish_po4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_p"] = fish_df["fish_p"] / 1000.0  # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4
ospar_region,,,,,
NORWAY,0.0,0.0,0.0,0.0,0.0
LOFOTEN-BARENTS SEA,0.0,0.0,0.0,0.0,0.0
NORTH SEA,0.0,0.0,0.0,0.0,0.0
NORWEGIAN SEA2,0.0,0.0,0.0,0.0,0.0
SKAGERAK,0.0,0.0,0.0,0.0,0.0


In [14]:
# Update sheet 5c
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total", "Cu"]

update_spreadsheet_point_sources(temp_path, "5c", pars, "fish", fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [15]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split("_")[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby("region").sum()

td_df.columns = ["td_" + i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.000,0.000,0.0,0.0000,0.00000,0.000,0.0000,1600.0000,0.0,0.000,0.000,0.00000,0.00000,0.000000,0.000000
NORTH SEA,0.000,1.900,0.4,3.9980,0.03810,1.300,3.1630,10.1290,178.6,92.244,0.132,0.09900,0.00660,0.000283,0.000170
NORWAY,0.485,1.925,0.4,21.9958,0.04648,5.456,3.9523,1622.1882,395.7,95.169,2.962,2.22150,0.14810,0.155525,0.093315
NORWEGIAN SEA2,0.000,0.000,0.0,0.0000,0.00000,0.000,0.0000,5.1020,140.0,0.000,0.909,0.68175,0.04545,0.064620,0.038772
SKAGERAK,0.485,0.025,0.0,17.9978,0.00838,4.156,0.7893,6.9572,77.1,2.925,1.921,1.44075,0.09605,0.090622,0.054373


In [16]:
# Update sheet 5e
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5e", pars, "td", td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [17]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby("ospar_region").sum()
trib_df = mon_df[mon_df["old_rid_group"] != "rid_11"].groupby("ospar_region").sum()

In [18]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """Update the OSPAR template for monitored rivers.

    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "SPM",
        "TOC": "TOC",
        "PO4-P": "PO4-P",
        "P-Total": "TOTP",
        "NO3-N": "NO3-N",
        "NH4-N": "NH4-N",
        "N-Total": "TOTN",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names and rows
    names_dict = {
        "Norwegian Sea (NO)": ("NORWEGIAN SEA2", "tot"),
        "Barents Sea (NO)": ("LOFOTEN-BARENTS SEA", "tot"),
        "Skagerrak (NO)": ("SKAGERAK", "tot"),
        "North Sea (NO)": ("NORTH SEA", "tot"),
        "Norway Total": ("NORWAY", "tot"),
        "Tributary Rivers - Norwegian Sea": ("NORWEGIAN SEA2", "trib"),
        "Tributary Rivers - Barents Sea": ("LOFOTEN-BARENTS SEA", "trib"),
        "Tributary Rivers - Skagerak": ("SKAGERAK", "trib"),
        "Tributary Rivers - North Sea": ("NORTH SEA", "trib"),
        "Orkla": (29778, "main"),
        "Vefsna": (29782, "main"),
        "Alta": (29779, "main"),
        "Glomma": (29617, "main"),
        "Drammenselva": (29612, "main"),
        u"Numedalslågen": (29615, "main"),
        "Skienselva": (29613, "main"),
        "Otra": (29614, "main"),
        "Orreelva": (29783, "main"),
        "Vosso": (29821, "main"),
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]

        # Get df
        df = df_dict[df_name]

        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [19]:
# Update sheet 6a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]
df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6a", pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [20]:
# Get diff data
cols = [i for i in umon_df.columns if i.split("_")[0] == "diff"]
diff_df = umon_df[cols].copy()

# Convert units
diff_df["diff_nh4"] = diff_df["diff_nh4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_no3"] = diff_df["diff_no3"] / 1000.0  # tonnes to ktonnes
diff_df["diff_n"] = diff_df["diff_n"] / 1000.0  # tonnes to ktonnes
diff_df["diff_po4"] = diff_df["diff_po4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_p"] = diff_df["diff_p"] / 1000.0  # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,28.669823,0.610079,0.150079,17.918639,1.576840
LOFOTEN-BARENTS SEA,4.974577,0.092397,0.022730,3.109110,0.273602
NORTH SEA,11.051026,0.183404,0.045117,6.906891,0.607806
NORWEGIAN SEA2,9.439302,0.225178,0.055394,5.899564,0.519162
SKAGERAK,3.204918,0.109099,0.026838,2.003074,0.176271


In [21]:
# Update sheet 6b
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total"]

update_spreadsheet_point_sources(temp_path, "6b", pars, "diff", diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [22]:
# Rename cols in diff_df
col_map = {
    "diff_n": "TOTN",
    "diff_p": "TOTP",
    "diff_po4": "PO4-P",
    "diff_no3": "NO3-N",
    "diff_nh4": "NH4-N",
}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6c", pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [23]:
## Standardise col names
# diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
# td_df.columns = [i.split('_')[1] for i in td_df.columns]
# td_df.index.name = 'ospar_region'
#
# col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
# for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#
## Reset index
# diff_df.reset_index(inplace=True)
# tot_df.reset_index(inplace=True)
# td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
# tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
# tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
# tot_df

In [24]:
## Update sheet 6c
# pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N',
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P',
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
# update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [25]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/concs_and_flows_rid_11_{year}.csv"
conc_df = pd.read_csv(in_csv, index_col=0, encoding="utf-8")

cols = [
    "Hg_ng/l",
    "NH4-N_µg/l N",
    "NO3-N_µg/l N",
    "TOTN_µg/l N",
    "TOTP_µg/l P",
    "PO4-P_µg/l P",
    "TOC_mg C/l",
]
for col in cols:
    if col not in conc_df.columns:
        par = col.split("_")[0]
        conc_df[col] = np.nan
        conc_df[f"{par}_flag"] = np.nan

# Convert units
conc_df["Hg_ng/l"] = conc_df["Hg_ng/l"] / 1000  # ng to ug
conc_df["NH4-N_µg/l N"] = conc_df["NH4-N_µg/l N"] / 1000  # ug to mg
conc_df["NO3-N_µg/l N"] = conc_df["NO3-N_µg/l N"] / 1000  # ug to mg
conc_df["TOTN_µg/l N"] = conc_df["TOTN_µg/l N"] / 1000  # ug to mg
conc_df["TOTP_µg/l P"] = conc_df["TOTP_µg/l P"] / 1000  # ug to mg
conc_df["PO4-P_µg/l P"] = conc_df["PO4-P_µg/l P"] / 1000  # ug to mg
conc_df["TOC_mg C/l"] = conc_df["TOC_mg C/l"] * 1000  # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split("_")[1] == "flag"]
lod_df = conc_df[cols]
lod_df.columns = [i.split("_")[0] for i in lod_df.columns]

# Get vals
cols = [
    i
    for i in conc_df.columns
    if ((i.split("_")[0] in lod_df.columns) and (i.split("_")[1] != "flag"))
]
conc_df = conc_df[cols]
conc_df.columns = [i.split("_")[0] for i in conc_df.columns]

# Rename
col_dict = {
    "SPM": "S.P.M.",
    "TOTN": "n",
    "NH4-N": "nh4",
    "NO3-N": "no3",
    "TOTP": "p",
    "PO4-P": "po4",
}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {
    "SPM": "S.P.M.",
    "TOC": "TOC",
    "PO4-P": "po4",
    "P-Total": "p",
    "NO3-N": "no3",
    "NH4-N": "nh4",
    "N-Total": "n",
    "As": "As",
    "Pb": "Pb",
    "Cd": "Cd",
    "Cu": "Cu",
    "Zn": "Zn",
    "Ni": "Ni",
    "Total Cr": "Cr",
    "Hg": "Hg",
}

# Map names to stns
names_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Alta": 29779,
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Orreelva": 29783,
    "Vosso": 29821,
}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb["7"]

# Get row numbers
row_dict = {}
for item in ws["B12" : "B%s" % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws["E9":"AK9"][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        if par_dict[par] not in conc_df.columns:
            conc_df[par_dict[par]] = np.nan
            lod_df[par_dict[par]] = np.nan

        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].values
        lods = lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]].fillna("0").values
        vals[(lods == "<")] = 0
        val = np.nanmean(vals)

        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].dropna())
        if n_samp == 0:
            n_samp = np.nan
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]])).sum()
        try:
            pct_lod = 100 * float(n_lod) / float(n_samp)
            if pct_lod < 30:
                val = "Yes"
            else:
                val = "No"
        except ZeroDivisionError:
            val = np.nan

        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 